In [ ]:
!curl -o hello.zip https://p5awqg.bn.files.1drv.com/y4mA-IEUekCe26UIJgz2ProYp4zvREkBRgWqgnpQ-CG1ogbzKqNUbf_okXphIRbqldqeOqjSn0jwkvV5LSbzBMylwoFNyA4tujk9iuT3QMASbP_BFRNbw4sE1AuDtduNKpuImWahZ9--9kftzW27Dryb3wqoc6ba4_KZwgBe75nCNekCKIYV4-vUXiUBEffjfIccX6vuipt7_iy2SStMBAIgw

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48.0M  100 48.0M    0     0  6335k      0  0:00:07  0:00:07 --:--:-- 10.7M


In [ ]:
!pip3 install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [ ]:
import os
from collections import Counter
from statistics import median
from pprint import pprint
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

import nltk
from nltk import WordNetLemmatizer, PorterStemmer
nltk.download('wordnet')

import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW, Adam

from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import evaluate

print(f'Pytorch version: {torch.__version__}')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    print(f'Device: {torch.cuda.get_device_name(torch.cuda.current_device())}')

RANDOM_SEED = 42


[nltk_data] Downloading package wordnet to /root/nltk_data...


Pytorch version: 2.0.1+cu118
Device: Tesla T4


In [ ]:
!unzip hello.zip

Archive:  hello.zip
 extracting: ILDC_single/ILDC_single.csv.zip  
 extracting: ILDC_single/README.md   


In [ ]:
!unzip ILDC_single/ILDC_single.csv.zip

Archive:  ILDC_single/ILDC_single.csv.zip
  inflating: ILDC_single.csv         
   creating: __MACOSX/
  inflating: __MACOSX/._ILDC_single.csv  


In [ ]:
class Config():
    def __init__(self, filePath: str, baseModelName: str):
        self.filePath = filePath
        # self.MAX_TOKENS = 16384
        self.MAX_TOKENS = 1024
        self.BATCH_SIZE = 1
        self.baseModelName = baseModelName
        self.EPOCHS = 2
        self.tokenizer = AutoTokenizer.from_pretrained(baseModelName)

# config = Config('./ILDC_single.csv',
#                 'hyperonym/xlm-roberta-longformer-base-16384')
config = Config('./ILDC_single.csv',
                'allenai/longformer-base-4096')

In [ ]:
print(f'Unknown token: {config.tokenizer.unk_token}, ID: {config.tokenizer.unk_token_id}')
print(f'Seperator token: {config.tokenizer.sep_token}, ID: {config.tokenizer.sep_token_id}')
print(f'Padding token: {config.tokenizer.pad_token}, ID: {config.tokenizer.pad_token_id}')
print(f'mask token: {config.tokenizer.mask_token}, ID: {config.tokenizer.mask_token_id}')
print(f'Sentence level classification token: {config.tokenizer.cls_token}, ID: {config.tokenizer.cls_token_id}')

Unknown token: <unk>, ID: 3
Seperator token: </s>, ID: 2
Padding token: <pad>, ID: 1
mask token: <mask>, ID: 50264
Sentence level classification token: <s>, ID: 0


In [ ]:
data = pd.read_csv(config.filePath)
data.head()

,text,label,split,name
0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train,2019_890.txt
1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train,2014_170.txt
2,"Markandey Katju, J. Leave granted. Heard lear...",1,train,2010_721.txt
3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train,2008_1460.txt
4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train,2008_188.txt


In [ ]:
# data split percentages
data_s = Counter(data['split'])
num_rows = sum(data_s.values())
[float(f'{val/num_rows:.2f}') for val in data_s.values()]

[0.67, 0.2, 0.13]

In [ ]:
train_ds = data[data['split'] == 'train'].drop(['split', 'name'], axis=1)
dev_ds = data[data['split'] == 'dev'].drop(['split', 'name'], axis=1)
test_ds = data[data['split'] == 'test'].drop(['split', 'name'], axis=1)

del data

print(Counter(train_ds['label']))
print(Counter(dev_ds['label']))
print(Counter(test_ds['label']))

Counter({0: 3147, 1: 1935})
Counter({0: 497, 1: 497})
Counter({1: 762, 0: 755})


In [ ]:
import string

class LegalDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[float]) -> None:
        super(LegalDataset, self).__init__()
        self.texts = texts
        self.labels = labels
        self.lemmatizer = WordNetLemmatizer()
        self.porter = PorterStemmer()

    def __preprocess_text(self, data: str) -> str:
        text: str = data.strip()
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = self.lemmatizer.lemmatize(text, 'v')
        text = self.porter.stem(text)
        return text

    def __len__(self) -> int: return len(self.texts)

    def __getitem__(self, ix: int)-> dict[str, torch.Tensor]:
        text = self.texts[ix]
        # text = self.__preprocess_text(text)
        label = torch.tensor(self.labels[ix], dtype=torch.float16)

        inputs = config.tokenizer(
            text=text,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=config.MAX_TOKENS,
            return_tensors='pt'
        )

        inputs = {k: v[0] for k, v in inputs.items()}
        inputs['labels'] = label.type(torch.float32)
        return inputs


In [ ]:
hg_train_ds = LegalDataset(texts=train_ds['text'], labels=train_ds['label'])
hg_dev_ds = LegalDataset(texts=dev_ds['text'], labels=dev_ds['label'])

train_loader = DataLoader(hg_train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(hg_dev_ds, batch_size=config.BATCH_SIZE)

In [ ]:
del train_ds
del dev_ds

In [ ]:
batch = next(iter(train_loader))
{k:v for k, v in batch.items()}

{'input_ids': tensor([[   0,  347, 6372,  ...,  488,  223,    2]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([1.])}

In [ ]:
model = AutoModelForSequenceClassification\
        .from_pretrained(config.baseModelName, from_tf=True, num_labels=1)

All TF 2.0 model weights were used when initializing LongformerForSequenceClassification.

All the weights of LongformerForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForSequenceClassification for predictions without further training.


In [ ]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

5377035264
6348079104


In [ ]:
x = model.forward(**{k: v for k, v in batch.items()}, return_dict=True)

In [ ]:
x

LongformerSequenceClassifierOutput(loss=tensor(1.3237, grad_fn=<MseLossBackward0>), logits=tensor([[-0.1505]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)

In [ ]:
a, b = x.items()
a, b

(('loss', tensor(1.3237, grad_fn=<MseLossBackward0>)),
 ('logits', tensor([[-0.1505]], grad_fn=<AddmmBackward0>)))

In [ ]:
optimizer = Adam(params=model.parameters(), lr=3e-2)
TRAIN_STEPS = len(train_loader) * config.BATCH_SIZE

lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=TRAIN_STEPS
)

In [ ]:
if device == 'cuda': torch.cuda.empty_cache()

In [ ]:
class Trainer:
    def __init__(self, epochs: int) -> None:
        self.device = torch.device(
            "cuda" if torch.has_cuda else "mps" if torch.has_mps else "cpu"
        )
        self.epochs = epochs
        self.model = model.to(self.device)
        self.optimizer = Adam(model.parameters(), lr=3e-2)

        num_train_steps = len(train_loader) * config.BATCH_SIZE

        self.scheduler = get_scheduler(
            name='linear',
            optimizer=optimizer,
            num_warmup_steps=0,
            num_training_steps=num_train_steps
        )

        self.train_loader = None
        self.valid_loader = None

        self.best_model = {'loss': np.inf, 'weights': None}

        self.history = {
            'epoch': [],
            'train_loss': [],
            'valid_loss': []
        }


    def train_one_step(self, batch: Dict[str, torch.Tensor]) -> torch.Tensor:
        for k, v in batch.items():
            batch[k] = v.to(self.device)
        (_, loss), yhat = self.model.forward(**batch).items()
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.scheduler.step()
        return loss


    def train_one_epoch(self, data_loader: DataLoader) -> torch.Tensor:
        self.model.train()
        epoch_loss = 0.0
        for batch in tqdm(self.train_loader):
            loss = self.train_one_step(batch)
            epoch_loss += loss.item()
        print('training done')
        return epoch_loss / len(self.train_loader)


    def validate(self, data_loader: DataLoader) -> torch.Tensor:
        self.model.eval()
        total_loss = 0.0
        with torch.no_grad():
            for batch in data_loader:
                for k, v in batch.items():
                    batch[k] = v.to(self.device)

                loss, yHat = self.model.forward(**batch).items()
                total_loss += loss.item()

        print('validation done')

        return total_loss / len(data_loader)


    def predict(self, input_ids: torch.Tensor,
                token_type_ids: torch.Tensor,
                attention_mask: torch.Tensor) -> np.ndarray:
        yHat, _ = self.model.forward(input_ids=input_ids.to(self.device),
                                     token_type_ids=token_type_ids.to(self.device),
                                     attention_mask=attention_mask.to(self.device))
        return yHat.detach().cpu().numpy()


    def fit(self, train_loader: DataLoader, valid_loader: Optional[DataLoader] = None) -> None:

        if self.train_loader is None and train_loader is None:
            raise Exception("Train Loader cannot be None.")

        self.train_loader = train_loader
        self.valid_loader = valid_loader

        for epoch in range(self.epochs):
            print(f'Training on Epoch #[{epoch + 1}/{self.epochs}]')

            train_loss = self.train_one_epoch(self.train_loader)
            if self.valid_loader:
                valid_loss = self.validate(self.valid_loader)

            self.history['epoch'].append(epoch)
            self.history['train_loss'].append(train_loss)
            self.history['valid_loss'].append(valid_loss)

            if valid_loss < self.best_model['loss']:
                self.best_model['loss'] = valid_loss
                self.best_model['weights'] = self.model.state_dict()

            torch.save(self.model, 'model_{epoch}.pth')

            print(f"Epoch[{epoch+1}/{self.epochs}]train_loss: {self.history['train_loss'][-1]}\t valid_loss: {self.history['valid_loss'][-1]}", end='\r')

In [ ]:
trainer = Trainer(epochs=config.EPOCHS)

In [ ]:
trainer.fit(train_loader=train_loader, valid_loader=dev_loader)

Training on Epoch #[1/2]


  0%|          | 0/5082 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(trainer.history['epoch'], trainer.history['train_loss'], label='Train')
plt.plot(trainer.history['epoch'], trainer.history['valid_loss'], label='Valid')
plt.legend()
plt.show()

In [1]:
trainer.best_model

NameError: name 'trainer' is not defined

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
torch.save(trainer.best_model, 'best_model.pth')

In [ ]:
def train_model(model,
                train_dataloader,
                epochs,
                train_steps,
                optimizer,
                lr_scheduler):

    progress_bar = tqdm(range(train_steps))
    losses = []

    model.train()
    for epoch in range(epochs):
        for batch in train_dataloader:
            # get batch data
            print(batch.items())
            batch = {k: v.to(device) for k, v in batch.items()}
            print(batch)

            # # get model output
            outputs = model(**batch)

            # # get loss and backprop
            # loss = outputs.loss
            # losses.append(loss)

            # loss.backward()
            # optimizer.step()
            # lr_scheduler.step()

            # # clear gradients
            # optimizer.zero_grad()

            # # update progress
            # progress_bar.update(1)

In [ ]:
# !export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
import torch
torch.cuda.empty_cache()

EPOCHS = 2

train_model(model,
            dev_dl,
            EPOCHS,
            TRAIN_STEPS,
            optimizer,
            lr_scheduler)